In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Part 1: Prepare the Data

In [ ]:
# 1.1 Read myopia.csv into a Pandas DataFrame.
file_path = Path("../Resources/Myopia.csv")
df = pd.read_csv(file_path)
df.head()

In [ ]:
# 1.2 Remove the "MYOPIC" column from the dataset.
df.drop("MYOPIC", axis=1)

In [ ]:
# 1.3 Standardize your dataset so that columns that contain larger values do not influence 
# the outcome more than columns with smaller values.


# Part 2: Apply Dimensionality Reduction

In [ ]:
# 2.1 Perform dimensionality reduction with PCA. How did the number of the features change?

In [ ]:
# 2.2 Further reduce the dataset dimensions with t-SNE and visually inspect the results. 
# To do this, run t-SNE on the principal components, which is the output of the PCA transformation.

In [ ]:
# 2.3 Create a scatter plot of the t-SNE output. Are there distinct clusters?

# Part 3: Perform a Cluster Analysis with K-means

In [ ]:
#3.1  Create an elbow plot to identify the best number of clusters. Make sure to do the following:
# Use a for loop to determine the inertia for each k between 1 through 10.


# 3.2 If possible, determine where the elbow of the plot is, and at which value of k it appears.

# Part 4: Make a Recommendation